In [ ]:
#!wget http://stgr1.sdp.mkat.chpc.kat.ac.za:7480/1539273852/1539273852_sdp_l0.rdb
!wget http://stgr1.sdp.mkat.chpc.kat.ac.za:7480/1539354654/1539354654_sdp_l0.full.rdb

# AR1 Observation report

## File to be processed

In [ ]:

#filename="Whatever"
filename="1539354654_sdp_l0.full.rdb"


## Set up environment

In [ ]:
%pylab inline

In [ ]:
import katdal
import scape
import pickle
import katpoint

## Load static flags

In [ ]:
#pickle_file = open('/var/kat/katsdpscripts/RTS/rfi_mask.pickle')
pickle_file = open('/var/kat/katsdpscripts/RTS/rfi_mask.pickle')
rfi_static_flags = pickle.load(pickle_file)
pickle_file.close()
edge = np.tile(True,4096)
edge[slice(211,3896)] = False
static_flags = np.logical_or(edge,rfi_static_flags)

## Load data

In [ ]:
data = katdal.open(filename)

In [ ]:
N = data.shape[0]
ext = 930
step = max(int(floor(N/ext)),1)
print step

In [ ]:
print data

In [ ]:
data.obs_params

In [ ]:
data.timestamps[1] - data.timestamps[0]

In [ ]:
import logging
logging.root.setLevel('INFO')

In [ ]:
data.select()
ants = data.ants
for ant in ants:
    print ant.description

In [ ]:
for active_pol in ['H','V']:
    print "Doing pol",active_pol
    data.select()
    data.select(corrprods='cross', channels=slice(2200,2800),pol=active_pol,scans=['scan','track'])
    baseline_names = [('%s - %s' % (inpA[:-1], inpB[:-1])) for inpA, inpB in data.corr_products]
    num_bls = data.shape[2]
    #num_bls=0
    num_chans, sample_period = len(data.channel_freqs), data.dump_period
    scan_targets, scan_mid_az, scan_mid_el, scan_timestamps, scan_phase = [], [], [], [], []
    for scan_ind, state, target in data.scans():
        num_ts = data.shape[0]
        ts = data.timestamps[0::step]
        vis = data.vis[0::step,:,:]
        scan_phase.append(np.angle(vis).T.reshape(-1, vis.shape[0]))
        scan_timestamps.append(ts - data.start_time.secs)
    scan_freqinds = [np.arange(num_bls * num_chans)] * len(scan_timestamps)
    print "Debug", num_bls,num_chans,len(scan_timestamps),"\nxxxxxxxxxxxxxxxxxxx"
    figure(figsize=(15,200))
    segms, labels, lines = scape.plots_basic.plot_segments(scan_timestamps, scan_freqinds,\
                                scan_phase, labels=scan_targets)
    for label in labels:
        label.set_rotation('vertical')
    plt.xlabel('Time (s), since %s' % (katpoint.Timestamp(data.start_time).local(),))
    plt.yticks(np.arange(num_chans // 2, num_bls * num_chans, num_chans), baseline_names,ha='right',y=1.0,fontsize=7)
    for yval in range(0, num_bls * num_chans, num_chans):
        plt.axhline(yval, color='k', lw=1, alpha=0.1)
        #plt.axvline(yval, color='y', lw=1, alpha=0.1)
    plt.title('Raw visibility phase per baseline, Pol %s'%active_pol)

In [ ]:
print 'Report generated:'
!date